# Capstone Project

## Neighbourhood Analysis for City of Toronto

## Summary:
This notebook will be used for Capstone Project for "IBM Data Science Professional Certificate" course on coursera. Location data is analysed to gain insights on the neighbourhoods in the specified area. In this notebook, we will explore, segment, and cluster the neighbourhoods in the city of <b>Toronto</b>.


In [27]:
#import required libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [28]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Capture neighbourhoods in the city of Toronto
Neighborhood details for the city of Torontaby are available on it's wiki page <a>https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050</a>.
We need to scrape the wiki page and extract required information from it.

In [29]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'

df = pd.read_html(url, match='Postcode', header=0)[0] # read an html table that contains 'Postcode' and load it into a dataframe
print (df.shape)
df.head()

(287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Preprocess data

Only process the cells that have an assigned Borough. Ignore cells with a borough that is Not assigned.

In [30]:

df['Borough'].replace('Not assigned', np.nan, inplace=True) # replace 'Not assigned' with NAN
df.dropna(axis=0, inplace=True) # drop rows where NAN
print (df.shape)
df.head()

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area
Combine neighbourhood with comma for each postal code. 
Use Groupby on Postcode and Borough and apply String join function on Neigbourhood (Refer w3schools python string join tutorial)

In [31]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
print (df.shape)
df.head()

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a Borough but a Not assigned Neighborhood, then the Neighborhood will be the same as the borough

In [32]:
df['Neighbourhood'].replace('Not assigned', np.nan, inplace=True) # replace 'Not assigned' with NAN
df['Neighbourhood'].fillna(df['Borough'], inplace=True) # Replace with Borough
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [33]:
print ('The Neighbourhood dataframe has {} number of rows'.format(df.shape[0]))

The Neighbourhood dataframe has 103 number of rows


### Identify geographical coordinates for the neighbourhoods

A csv file that has the geographical coordinates of each postal code can be found at: http://cocl.us/Geospatial_data

In [34]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data') # read csv having coordinate for postal code
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the toronto neighbourhood data with geo data using merge function on postal code

In [35]:
df_with_geo_data = pd.merge(df, geo_data, left_on='Postcode',right_on='Postal Code', how='inner') 
df_with_geo_data.drop(['Postal Code'], axis=1, inplace=True) # drop duplicate postal code column
df_with_geo_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Work with Boroughs that contain Toronto

In [36]:
Toronto_df = df_with_geo_data[df_with_geo_data['Borough'].str.contains("Toronto")].reset_index(drop=True) #Filter borough column that contains 'Toronto'
Toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Define Foursquare API credentials and version
Foursquare is a social location service that allows users to explore the world around them. Based on lattitude and longitude, it will provide nearby services such as hotels, banks, stores, specific type of restaurants etc.


In [37]:
# @hidden_cell
CLIENT_ID = 'IES25B4UWY215FFF3QSWXE5TAAQYYB4RJCTKESOXNA0PUYGA' # your Foursquare ID
CLIENT_SECRET = 'M5ZPM1KX30PYQVQ3UW1ZRA4IIUHLVZVOQP53VOZJSFXUWYWG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Explore the Toronto neighbourhood

#### Function to accept neighbourhood details and call Foursquare API to explore a location. Process the location data and store it in a list

In [38]:
# For each given lat,long, explore the venue using foursquare api
# Convert output of api response into json and store the required information in a list
def exploreNeighbourhood(names, latitudes, longitudes, RADIUS=500, LIMIT=30 ):
    venues_list=[]
    #For each selected neighbourhood, explore it using foursquare explore api
    for name,lat,long in zip(names,latitudes,longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
           CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,RADIUS,LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items'] # parse the result to get items/venues
        
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])   
      
    print ("Data fetched succesfully.")
    return (venues_list)

#### Explore 30 venues in distance of 500 meters for each neighbourhood using the exploreNeighbourhood function

In [39]:
exploration_results = exploreNeighbourhood(Toronto_df['Neighbourhood'],Toronto_df['Latitude'],Toronto_df['Longitude'],500,30)

Data fetched succesfully.


#### Convert exploration results into dataframe

In [40]:
Toronto_venues = pd.DataFrame([item for venue_list in exploration_results for item in venue_list]) # add each item in venue list to dataframe,
Toronto_venues.columns = ['Neighbourhood', 
              'Neighbourhood Latitude', 
              'Neighbourhood Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']
        
Toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Indentify unique venue categories

In [41]:
print ("There are {} number of unique venue categories in the neighbourhood.".format(len(Toronto_venues['Venue Category'].unique())))

There are 196 number of unique venue categories in the neighbourhood.


#### Indetify venue counts for each neighbourhood

In [42]:
Toronto_venues.groupby('Neighbourhood')['Venue'].count()

Neighbourhood
Adelaide,King,Richmond                                                                                  30
Berczy Park                                                                                             30
Brockton,Exhibition Place,Parkdale Village                                                              24
Business Reply Mail Processing Centre 969 Eastern                                                       18
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara    16
Cabbagetown,St. James Town                                                                              30
Central Bay Street                                                                                      30
Chinatown,Grange Park,Kensington Market                                                                 30
Christie                                                                                                18
Church and Wellesley   

#### Use One Hot Encoding on Venue Category to convert categorical values in columns and mark the venue in the appropriate column

In [43]:
dummies = pd.get_dummies(Toronto_venues['Venue Category']) # Execute one hot encode on 'Venue Category'
Toronto_venues_x = pd.concat([Toronto_venues,dummies],axis=1)
Toronto_venues_x.drop(['Venue Category','Neighbourhood Latitude','Neighbourhood Longitude','Venue Latitude','Venue Longitude',],axis=1,inplace=True) # remove 'Venue Category','Neighbourhood Latitude','Neighbourhood Longitude','Venue Latitude','Venue Longitude' column as it has been one hot encoded
Toronto_venues_x.head()

,Neighbourhood,Venue,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,Glen Manor Ravine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,The Beaches,The Big Carrot Natural Food Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,Grover Pub and Grub,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0

#### Find average of occurence of each category for each neighbourhood

In [44]:
Toronto_grouped = Toronto_venues_x.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.033333,0.000000,0.0,0.066667,0.0,0.000000,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.033333,0.0,0.0,0.0,0.033333,0.0,0.000000,0.033333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.033333,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333,0.000000,0.000000,0.033333,0.0000,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.000000,0.0,0.033333,0.0,0.000000,0.033333,0.033333,0.0,0.000000,0.000000,0.000000,0.0,0.033333,0.0000,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.033333,0.0,0.0,0.0,0.0000,0.066667,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.033333,0.0,0.0,0.000000,0.0,0.033333,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0
1,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0

#### Sort on the frequency of categories for each neighbourhood. Return top N values

Create a function for sorting

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10
# create columns according to number of top venues
columns = ['Neighbourhood'] 
for ind in np.arange(num_top_venues): #columns = 'Neighborhood', 'Top 1 Most Common Venue', 'Top 2 Most Common Venue' .., 'Top 10 Most
    columns.append('Top {} Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

#Iterate over each row and sort according to catergory frequency. Return top 10 catergories
for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,Top 1 Most Common Venue,Top 2 Most Common Venue,Top 3 Most Common Venue,Top 4 Most Common Venue,Top 5 Most Common Venue,Top 6 Most Common Venue,Top 7 Most Common Venue,Top 8 Most Common Venue,Top 9 Most Common Venue,Top 10 Most Common Venue
0,"Adelaide,King,Richmond",Café,Seafood Restaurant,Hotel,Asian Restaurant,Monument / Landmark,Gym / Fitness Center,Restaurant,Colombian Restaurant,Plaza,Smoke Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Farmers Market,Beer Bar,Bakery,Restaurant,Breakfast Spot,Fish Market
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Nightclub,Pet Store,Restaurant,Japanese Restaurant,Italian Restaurant,Intersection,Stadium
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Smoke Shop,Recording Studio,Brewery,Butcher,Restaurant,Auto Workshop,Spa,Farmers Market
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Airport,Coffee Shop,Boutique,Boat or Ferry,Rental Car Location,Plane,Harbor / Marina


### Cluster Neighbourhoods

#### Create N clusters for Toronto neihbourhood

In [47]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       4, 3, 1, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 0, 3, 3], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [48]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Top 1 Most Common Venue,Top 2 Most Common Venue,Top 3 Most Common Venue,Top 4 Most Common Venue,Top 5 Most Common Venue,Top 6 Most Common Venue,Top 7 Most Common Venue,Top 8 Most Common Venue,Top 9 Most Common Venue,Top 10 Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Neighborhood,Pub,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,3,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Café,Pub
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,3,Park,Fish & Chips Shop,Sandwich Place,Brewery,Restaurant,Liquor Store,Italian Restaurant,Board Shop,Pub,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,Bakery,Italian Restaurant,Pet Store,Bookstore,Seafood Restaurant,Brewery,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


#### Visualize the clusters

In [49]:
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto city'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
colors_array = ['red','blue','green','brown','purple']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors_array[cluster-1],
        fill=True,
        fill_color=colors_array[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters